In [1]:
import dask.dataframe as dd

from rxitect.generator.datamodules.chembl_datamodule import ChemblSmilesDataModule
from rxitect.generator.datamodules.components.molecule_dataset import MoleculeDataset
from rxitect.structs.vocabulary import SmilesVocabulary

/home/julius/miniconda3/envs/rxt-dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
voc = SmilesVocabulary(vocabulary_filepath="../data/processed/smiles_voc.txt")

In [5]:
import pandas as pd

data = pd.read_table("../data/processed/smiles_chembl_corpus.tsv")  # .head(n=1_870_000)
data

,smiles,token
0,CCOc1ccc2[nH]nc(-c3cc(N4CC(F)C(F)C4)ncn3)c2c1,C C O c 1 c c c 2 [nH] n c ( - c 3 c c ( N 4 C...
1,N=C(N)Nc1nnn(Cc2cccc(Cn3nnc(C(=N)N)n3)c2)n1,N = C ( N ) N c 1 n n n ( C c 2 c c c c ( C n ...
2,CCCCNC(=O)c1cnc(S)n1C(CC)c1ccc(F)c(F)c1,C C C C N C ( = O ) c 1 c n c ( S ) n 1 C ( C ...
3,CC(=O)Oc1ccc(I)cc1C(=O)Nc1ccc(Cl)cc1,C C ( = O ) O c 1 c c c ( I ) c c 1 C ( = O ) ...
4,COc1ccc(C(=O)N2CCN(Cc3ccc(-c4ccncc4)cc3)CC2)cc1,C O c 1 c c c ( C ( = O ) N 2 C C N ( C c 3 c ...
...,...,...
1871222,CC(C)c1ccc(-c2nc(SCc3cn(CC(=O)NC(=O)Nc4ccccn4)...,C C ( C ) c 1 c c c ( - c 2 n c ( S C c 3 c n ...
1871223,Nc1c(-c2csc(-c3ccncc3)n2)c(=O)[nH]c2ccc(N3CCCC...,N c 1 c ( - c 2 c s c ( - c 3 c c n c c 3 ) n ...
1871224,O=C(COC1CCCC1)NCCc1nc(-c2ccccn2)cs1,O = C ( C O C 1 C C C C 1 ) N C C c 1 n c ( - ...
1871225,CC(C)N1C(=O)CN(Cc2ccc(-c3ccc(CC(=O)O)cc3)cn2)C1=O,C C ( C ) N 1 C ( = O ) C N ( C c 2 c c c ( - ...


In [ ]:
ds = MoleculeDataset(data=data, vocabulary=voc)

In [ ]:
ds[0].reshape(-1).shape

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=128, pin_memory=True)

In [ ]:
test_batch = next(iter(dl))
test_batch

In [ ]:
reshaped = test_batch.reshape(128, 100)

In [ ]:
import matplotlib.pyplot as plt


plt.matshow(reshaped)

In [ ]:
from rdkit.Chem import PandasTools, Descriptors

PandasTools.AddMoleculeColumnToFrame(data,'smiles','ROMol')
data["MW"] = data.ROMol.apply(Descriptors.MolWt)
_ = plt.hist(data["MW"])

In [ ]:
mydm = ChemblSmilesDataModule("../data/processed/smiles_chembl_corpus.tsv", "../data/processed/smiles_voc.txt", train_val_test_split=(70_000, 10_000, 20_000))

In [ ]:
mydm.prepare_data()
mydm.setup()

In [ ]:
vdl = mydm.val_dataloader()

In [ ]:
val_batch = next(iter(vdl))
val_batch

In [ ]:
plt.matshow(val_batch)